# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
train_data = []
# train_data = []
num_cands = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 5)
max_session = metadata['max_session']


for i in tqdm(range(num_frags)):
    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))






100%|██████████| 5/5 [00:00<00:00,  5.60it/s]


In [3]:
train_data = pd.concat(train_data)

In [4]:
candidates = train_data
candidates.shape

(1734693, 161)

In [5]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [6]:
candidates

,user,item,consistency,degree,fitness,inter_carts,inter_clicks,inter_durability,inter_fts,inter_lts,inter_num_interacts,inter_num_sub,inter_orders,inter_time_decay,is_level1,item_degree,item_glob_first_action,item_glob_item_glob_durability,item_glob_last_action,item_glob_time_decay_sum,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_click,item_glob_time_decay_sum_order,item_pr,item_recent_degree,item_recent_pr,lincom_recent_day_1_10_1_3,lincom_recent_day_1_10_3_1,lincom_recent_day_1_1_10_3,lincom_recent_day_1_1_3_10,lincom_recent_day_1_3_10_1,lincom_recent_day_1_3_1_10,lincom_recent_day_2_10_1_3,lincom_recent_day_2_10_3_1,lincom_recent_day_2_1_10_3,lincom_recent_day_2_1_3_10,lincom_recent_day_2_3_10_1,lincom_recent_day_2_3_1_10,lincom_recent_day_3_10_1_3,lincom_recent_day_3_10_3_1,lincom_recent_day_3_1_10_3,lincom_recent_day_3_1_3_10,lincom_recent_day_3_3_10_1,lincom_recent_day_3_3_1_10,lincom_recent_day_4_10_1_3,lincom_recent_day_4_10_3_1,lincom_recent_day_4_1_10_3,lincom_recent_day_4_1_3_10,lincom_recent_day_4_3_10_1,lincom_recent_day_4_3_1_10,lincom_recent_day_5_10_1_3,lincom_recent_day_5_10_3_1,lincom_recent_day_5_1_10_3,lincom_recent_day_5_1_3_10,lincom_recent_day_5_3_10_1,lincom_recent_day_5_3_1_10,lincom_recent_day_6_10_1_3,lincom_recent_day_6_10_3_1,lincom_recent_day_6_1_10_3,lincom_recent_day_6_1_3_10,lincom_recent_day_6_3_10_1,lincom_recent_day_6_3_1_10,lincom_recent_day_7_10_1_3,lincom_recent_day_7_10_3_1,lincom_recent_day_7_1_10_3,lincom_recent_day_7_1_3_10,lincom_recent_day_7_3_10_1,lincom_recent_day_7_3_1_10,num_actions,num_carts,num_clicks,num_orders,num_sub,pr,qou_cocart_sub_coef_mean,qou_cocart_sub_coef_sqrt_num_cousers,qou_cocart_time_decay_mean,qou_cocart_time_decay_sqrt_num_cousers,qou_coclick_sub_coef_mean,qou_coclick_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_mean,qou_coclick_time_decay_sqrt_num_cousers,qou_coorder_sub_coef_mean,qou_coorder_sub_coef_sqrt_num_cousers,qou_coorder_time_decay_mean,qou_coorder_time_decay_sqrt_num_cousers,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_time_decay_3_1_10_mean,qou_num_appearance_mean,qou_num_appearance_sqrt_num_cousers,qou_num_cosub_mean,qou_num_cosub_sqrt_num_cousers,recent_day1_type0,recent_day1_type1,recent_day1_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_degree,recent_inter_day1_type0,recent_inter_day1_type1,recent_inter_day1_type2,recent_inter_day2_type0,recent_inter_day2_type1,recent_inter_day2_type2,recent_inter_day3_type0,recent_inter_day3_type1,recent_inter_day3_type2,recent_inter_day4_type0,recent_inter_day4_type1,recent_inter_day4_type2,recent_inter_day5_type0,recent_inter_day5_type1,recent_inter_day5_type2,recent_inter_day6_type0,recent_inter_day6_type1,recent_inter_day6_type2,recent_inter_day7_type0,recent_inter_day7_type1,recent_inter_day7_type2,recent_num_carts,recent_num_clicks,recent_num_orders,recent_pr,user_degree,user_pr,user_recent_degree,user_recent_pr,weight_covisit_1week,weight_covisit_day1,weight_covisit_day2,weight_covisit_day3,weight_covisit_day4,weight_covisit_day5,weight_covisit_day6,weight_covisit_day7
0,12019416.0,12908762.0,1.0,337.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.0,0.443194,20.497963,20.941158,296.096924,21.401438,265.345551,9.349930,1.330284e-06,192.0,1.622016e-06,1098.145508,506.838074,913.663818,2983.239746,363.352325,3024.235840,556.0,560.0,88.0,74.0,196.0,178.0,401.0,407.0,95.0,74.0,169.0,1

# Train params

In [15]:
from meow_utils import *
from catboost.utils import eval_metric

PARAMS = {
    'learning_rate': 0.5,
    'iterations': 1000,
    'early_stopping_rounds' : 20,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': 'RecallAt:top=20',
    'use_best_model': True,

}
N_SPLITS = 3
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '1'

# Train for each type

In [8]:
from meow_utils import get_len_group

In [9]:
candidates

,user,item,consistency,degree,fitness,inter_carts,inter_clicks,inter_durability,inter_fts,inter_lts,inter_num_interacts,inter_num_sub,inter_orders,inter_time_decay,is_level1,item_degree,item_glob_first_action,item_glob_item_glob_durability,item_glob_last_action,item_glob_time_decay_sum,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_click,item_glob_time_decay_sum_order,item_pr,item_recent_degree,item_recent_pr,lincom_recent_day_1_10_1_3,lincom_recent_day_1_10_3_1,lincom_recent_day_1_1_10_3,lincom_recent_day_1_1_3_10,lincom_recent_day_1_3_10_1,lincom_recent_day_1_3_1_10,lincom_recent_day_2_10_1_3,lincom_recent_day_2_10_3_1,lincom_recent_day_2_1_10_3,lincom_recent_day_2_1_3_10,lincom_recent_day_2_3_10_1,lincom_recent_day_2_3_1_10,lincom_recent_day_3_10_1_3,lincom_recent_day_3_10_3_1,lincom_recent_day_3_1_10_3,lincom_recent_day_3_1_3_10,lincom_recent_day_3_3_10_1,lincom_recent_day_3_3_1_10,lincom_recent_day_4_10_1_3,lincom_recent_day_4_10_3_1,lincom_recent_day_4_1_10_3,lincom_recent_day_4_1_3_10,lincom_recent_day_4_3_10_1,lincom_recent_day_4_3_1_10,lincom_recent_day_5_10_1_3,lincom_recent_day_5_10_3_1,lincom_recent_day_5_1_10_3,lincom_recent_day_5_1_3_10,lincom_recent_day_5_3_10_1,lincom_recent_day_5_3_1_10,lincom_recent_day_6_10_1_3,lincom_recent_day_6_10_3_1,lincom_recent_day_6_1_10_3,lincom_recent_day_6_1_3_10,lincom_recent_day_6_3_10_1,lincom_recent_day_6_3_1_10,lincom_recent_day_7_10_1_3,lincom_recent_day_7_10_3_1,lincom_recent_day_7_1_10_3,lincom_recent_day_7_1_3_10,lincom_recent_day_7_3_10_1,lincom_recent_day_7_3_1_10,num_actions,num_carts,num_clicks,num_orders,num_sub,pr,qou_cocart_sub_coef_mean,qou_cocart_sub_coef_sqrt_num_cousers,qou_cocart_time_decay_mean,qou_cocart_time_decay_sqrt_num_cousers,qou_coclick_sub_coef_mean,qou_coclick_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_mean,qou_coclick_time_decay_sqrt_num_cousers,qou_coorder_sub_coef_mean,qou_coorder_sub_coef_sqrt_num_cousers,qou_coorder_time_decay_mean,qou_coorder_time_decay_sqrt_num_cousers,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_time_decay_3_1_10_mean,qou_num_appearance_mean,qou_num_appearance_sqrt_num_cousers,qou_num_cosub_mean,qou_num_cosub_sqrt_num_cousers,recent_day1_type0,recent_day1_type1,recent_day1_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_degree,recent_inter_day1_type0,recent_inter_day1_type1,recent_inter_day1_type2,recent_inter_day2_type0,recent_inter_day2_type1,recent_inter_day2_type2,recent_inter_day3_type0,recent_inter_day3_type1,recent_inter_day3_type2,recent_inter_day4_type0,recent_inter_day4_type1,recent_inter_day4_type2,recent_inter_day5_type0,recent_inter_day5_type1,recent_inter_day5_type2,recent_inter_day6_type0,recent_inter_day6_type1,recent_inter_day6_type2,recent_inter_day7_type0,recent_inter_day7_type1,recent_inter_day7_type2,recent_num_carts,recent_num_clicks,recent_num_orders,recent_pr,user_degree,user_pr,user_recent_degree,user_recent_pr,weight_covisit_1week,weight_covisit_day1,weight_covisit_day2,weight_covisit_day3,weight_covisit_day4,weight_covisit_day5,weight_covisit_day6,weight_covisit_day7
0,12019416.0,12908762.0,1.0,337.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.0,0.443194,20.497963,20.941158,296.096924,21.401438,265.345551,9.349930,1.330284e-06,192.0,1.622016e-06,1098.145508,506.838074,913.663818,2983.239746,363.352325,3024.235840,556.0,560.0,88.0,74.0,196.0,178.0,401.0,407.0,95.0,74.0,169.0,1

In [10]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [11]:
num_cands = candidates[['user', 'item']].groupby('user').count().reset_index()
num_cands['num_cands'] = num_cands['item']
del num_cands['item'] 

num_cands = candidates.merge(num_cands,on=['user'],how='left')['num_cands'].values


In [12]:
labels = []
for i in range(3):
  tar_small = tar[tar['type'] == i]
  
  labels.append(candidates.merge(tar_small[['item', 'label', 'user']],on=['user','item'],how='left').fillna(0)['label'].values.astype(int))
  
  



In [13]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold
import os
import shutil
from catboost import CatBoostRanker, Pool, EShapCalcType, EFeaturesSelectionAlgorithm

def train(t, id_type):
  global candidates, num_cands, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  
  skf = GroupKFold(n_splits=N_SPLITS)
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):

      X_train = candidates.iloc[train_idx, 2:]
      y_train = labels[id_type][train_idx]
      X_valid = candidates.iloc[valid_idx, 2:]
      y_valid = labels[id_type][valid_idx]

      dtrain = Pool(X_train, y_train, thread_count = os.cpu_count(), group_id = candidates.iloc[train_idx, 0].astype(int)) 
      dvalid = Pool(X_valid, y_valid, thread_count = os.cpu_count(), group_id = candidates.iloc[valid_idx, 0].astype(int))
      
      params = {'thread_count' : os.cpu_count(),
      **PARAMS}
      model = CatBoostRanker(**params)
      model.fit(dtrain, eval_set = dvalid)
      try:
        os.mkdir(MODEL_PATH + name + VERSION)
      except:
        pass
      model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}')
  


In [16]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(t, id_type)

0:	learn: 0.8003000	test: 0.7933991	best: 0.7933991 (0)	total: 597ms	remaining: 9m 56s
1:	learn: 0.8339404	test: 0.8169739	best: 0.8169739 (1)	total: 900ms	remaining: 7m 29s
2:	learn: 0.8390829	test: 0.8229747	best: 0.8229747 (2)	total: 1.23s	remaining: 6m 47s
3:	learn: 0.8435826	test: 0.8251179	best: 0.8251179 (3)	total: 1.55s	remaining: 6m 25s
4:	learn: 0.8452968	test: 0.8264038	best: 0.8264038 (4)	total: 1.86s	remaining: 6m 10s
5:	learn: 0.8457253	test: 0.8268324	best: 0.8268324 (5)	total: 2.17s	remaining: 5m 58s
6:	learn: 0.8472252	test: 0.8285469	best: 0.8285469 (6)	total: 2.49s	remaining: 5m 53s
7:	learn: 0.8474395	test: 0.8272610	best: 0.8285469 (6)	total: 2.81s	remaining: 5m 48s
8:	learn: 0.8470109	test: 0.8276897	best: 0.8285469 (6)	total: 3.11s	remaining: 5m 42s
9:	learn: 0.8485108	test: 0.8289756	best: 0.8289756 (9)	total: 3.46s	remaining: 5m 42s
10:	learn: 0.8478680	test: 0.8302615	best: 0.8302615 (10)	total: 3.78s	remaining: 5m 39s
11:	learn: 0.8497964	test: 0.8311187	best

Training has stopped (degenerate solution on iteration 33, probably too small l2-regularization, try to increase it)



bestTest = 0.9926938542
bestIteration = 19

Shrink model to first 20 iterations.
0:	learn: 0.9916266	test: 0.9899294	best: 0.9899294 (0)	total: 315ms	remaining: 5m 14s
1:	learn: 0.9931322	test: 0.9911823	best: 0.9911823 (1)	total: 629ms	remaining: 5m 13s
2:	learn: 0.9933465	test: 0.9911966	best: 0.9911966 (2)	total: 930ms	remaining: 5m 9s
3:	learn: 0.9940430	test: 0.9912680	best: 0.9912680 (3)	total: 1.23s	remaining: 5m 6s
4:	learn: 0.9936144	test: 0.9910752	best: 0.9912680 (3)	total: 1.52s	remaining: 5m 2s
5:	learn: 0.9938287	test: 0.9912180	best: 0.9912680 (3)	total: 1.81s	remaining: 5m
6:	learn: 0.9938644	test: 0.9913251	best: 0.9913251 (6)	total: 2.11s	remaining: 4m 58s
7:	learn: 0.9939180	test: 0.9905539	best: 0.9913251 (6)	total: 2.4s	remaining: 4m 57s
8:	learn: 0.9940252	test: 0.9896970	best: 0.9913251 (6)	total: 2.67s	remaining: 4m 54s
9:	learn: 0.9943288	test: 0.9886259	best: 0.9913251 (6)	total: 2.95s	remaining: 4m 52s
10:	learn: 0.9943645	test: 0.9886259	best: 0.9913251 (6)